In [5]:
import pandas as pd
import numpy as np

In [ ]:
bakery_sales = pd.read_csv('bakery_sales')
Coffee_sales = pd.read_csv('Coffee_Shop_sales.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Coffee_Shop_sales.csv'

In [ ]:
#clean the unit_price column by removing the € sign and converting the column to float
bakery_sales["unit_price"] = (
    bakery_sales["unit_price"]
    .str.replace("€", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace('"', "", regex=False)
    .str.strip()
    .astype(float)
)
#rename the columns to match the target schema
bakery_sales = bakery_sales.rename(
    columns={
        "ticket_number": "transaction_id",
        "date": "transaction_date",
        "time": "transaction_time",
        "Quantity": "transaction_qty",
        "article": "product_detail",
    }
)
#add the missing columns
bakery_sales["store_id"] = None
bakery_sales["store_location"] = None
bakery_sales["product_id"] = None
bakery_sales["product_category"] = "Bakery"
bakery_sales["product_type"] = None
#reorder the columns
bakery_sales = bakery_sales[
    [
        "transaction_id",
        "transaction_date",
        "transaction_time",
        "transaction_qty",
        "store_id",
        "store_location",
        "product_id",
        "unit_price",
        "product_category",
        "product_type",
        "product_detail",
    ]
]


In [ ]:
bakery_sales.to_csv("bakery_sales_new.csv", index=False)

In [ ]:
unique_categories = Coffee_sales['product_category'].unique()
print(unique_categories)

In [ ]:
bakery_items = Coffee_sales[Coffee_sales['product_category'] == 'Bakery'][['product_type', 'product_detail']].drop_duplicates()
print(bakery_items.nunique())


In [ ]:
bakery_sales_new = pd.read_csv("bakery_sales_new.csv")
bakery_items_new = bakery_sales_new[bakery_sales_new['product_category'] == 'Bakery'][['product_detail']].drop_duplicates()
print(bakery_items_new['product_detail'].unique())


In [ ]:
def map_article_to_category(article):
    normalized = article.strip().upper()
    
    if "CAFE" in normalized:
        return "Coffee", "Coffee", article
    if normalized == "THE":
        return "Tea", "Tea", article
    
    category = "Bakery"
    
    breads = ['BAGUETTE', 'PAIN', 'BOULE', 'PAIN DE MIE', 'SEIGLE', 'VIK BREAD', 'PAIN BANETTE', 'PAIN NOIR']
    viennoiseries = ['PAIN AU CHOCOLAT', 'CROISSANT', 'KOUIGN AMANN', 'BRIOCHE', 'CHAUSSON', 'VIENNOISERIE']
    patisseries = ['TARTE', 'ECLAIR', 'FRAISIER', 'ST HONORE', 'MACARON', 'ENTREMETS', 'PARIS BREST', 'ROYAL', 'TROPEZIENNE',
                   'FINANCIER', 'CRUMBLE', 'MILLES FEUILLES', 'RELIGIEUSE', 'MERINGUE']
    sandwiches = ['SANDWICH', 'SAND JB', 'FORMULE SANDWICH']
    beverages = ['BOISSON', 'EAU']
    meals = ['PLAT', 'TRAITEUR', 'PLATEAU', 'FORMULE PLAT PREPARE', 'PLATPREPARE']
    confections = ['CONFISERIE', 'SUCETTE', 'CHOCOLAT', 'MERINGUE', 'SABLE', 'BONBON']
    
    if any(word in normalized for word in breads):
        prod_type = "Bread"
    elif any(word in normalized for word in viennoiseries):
        prod_type = "Viennoiserie"
    elif any(word in normalized for word in patisseries):
        prod_type = "Pâtisserie"
    elif any(word in normalized for word in sandwiches):
        prod_type = "Sandwich"
    elif any(word in normalized for word in meals):
        prod_type = "Meal/Traiteur"
    elif any(word in normalized for word in beverages):
        prod_type = "Beverage"
    elif any(word in normalized for word in confections):
        prod_type = "Confectionery"
    else:
        prod_type = "Other"
        
    return category, prod_type, article

bakery_sales_new[['product_category', 'product_type', 'product_detail']] = bakery_sales_new['product_detail'].apply(
    lambda x: pd.Series(map_article_to_category(x))
)

bakery_sales_new.to_csv("bakery_sales_merged.csv", index=False)


In [ ]:
import pandas as pd

# Step 1: Create a mapping of product names to product IDs.
raw_product_names = [
    'BAGUETTE', 'PAIN AU CHOCOLAT', 'PAIN', 'TRADITIONAL BAGUETTE', 'CROISSANT',
    'BANETTE', 'BANETTINE', 'SPECIAL BREAD', 'COUPE', 'SAND JB EMMENTAL',
    'KOUIGN AMANN', 'BOULE 200G', 'BOULE 400G', 'GAL FRANGIPANE 6P', 'CAMPAGNE',
    'MOISSON', 'CAFE OU EAU', 'BRIOCHE', 'CEREAL BAGUETTE', 'SEIGLE', 'COMPLET',
    'DIVERS PATISSERIE', 'GAL FRANGIPANE 4P', 'COOKIE', 'FICELLE',
    'PAIN AUX RAISINS', 'GAL POMME 6P', 'GAL POMME 4P', 'FINANCIER X5',
    'VIK BREAD', 'DIVERS VIENNOISERIE', 'GACHE', 'SANDWICH COMPLET',
    'PAIN BANETTE', 'GRAND FAR BRETON', 'QUIM BREAD', 'SPECIAL BREAD KG',
    'GD KOUIGN AMANN', 'BOULE POLKA', 'DEMI BAGUETTE', 'CHAUSSON AUX POMMES',
    'BAGUETTE GRAINE', 'DIVERS CONFISERIE', 'SUCETTE', 'DIVERS BOULANGERIE',
    'BOISSON 33CL', 'PATES', 'FORMULE SANDWICH', 'DIVERS SANDWICHS',
    'CROISSANT AMANDES', 'PAIN CHOCO AMANDES', 'SACHET VIENNOISERIE', 'NANTAIS',
    'CHOCOLAT', 'PAIN S/SEL', 'FONDANT CHOCOLAT', 'GAL POIRE CHOCO 6P',
    'GAL POIRE CHOCO 4P', 'GALETTE 8 PERS', 'SAND JB', 'SACHET DE CROUTON',
    'GRANDE SUCETTE', 'DEMI PAIN', 'TARTELETTE', 'FLAN', 'PARIS BREST', 'SAVARIN',
    'FLAN ABRICOT', 'BAGUETTE APERO', 'MILLES FEUILLES', 'CHOU CHANTILLY',
    'ECLAIR', 'ROYAL 4P', 'TARTE FRUITS 6P', 'TARTE FRUITS 4P', 'NOIX JAPONAISE',
    'THE', 'BRIOCHETTE', 'ROYAL 6P', 'ECLAIR FRAISE PISTACHE', '.',
    'GD FAR BRETON', 'TRIANGLES', 'TROPEZIENNE', 'TROPEZIENNE FRAMBOISE', 'ROYAL',
    'TARTE FRAISE 6P', 'TARTELETTE FRAISE', 'TARTE FRAISE 4PER', 'FRAISIER',
    'NID DE POULE', 'TARTELETTE CHOC', 'PAIN DE MIE', 'CRUMBLE', 'FINANCIER',
    'DIVERS BOISSONS', 'CAKE', 'VIENNOISE', 'TRAITEUR', 'PAIN GRAINES',
    'PLATPREPARE6,50', 'PLATPREPARE5,50', 'PLATPREPARE7,00',
    'FORMULE PLAT PREPARE', 'ST HONORE', 'BROWNIES', 'RELIGIEUSE',
    'PLATPREPARE6,00', 'DELICETROPICAL', 'CRUMBLECARAMEL OU PISTAE',
    'PT NANTAIS', 'GD NANTAIS', 'DOUCEUR D HIVER', 'TROIS CHOCOLAT',
    'ARTICLE 295', 'TARTE FINE', 'ENTREMETS', 'BRIOCHE DE NOEL', 'FRAMBOISIER',
    'BUCHE 4PERS', 'BUCHE 6PERS', 'GD PLATEAU SALE', 'BUCHE 8PERS',
    'PT PLATEAU SALE', 'REDUCTION SUCREES 12', 'PAIN NOIR',
    'REDUCTION SUCREES 24', 'BOTTEREAU', 'MERINGUE', 'PALMIER', 'PAILLE',
    'PLAT 6.50E', 'PLAT 7.60E', 'PLAT 7.00', 'PLAT', 'PLAT 8.30E', 'FORMULE PATE',
    'GUERANDAIS', 'PALET BRETON', 'CARAMEL NOIX', 'MACARON', '12 MACARON',
    'ARMORICAIN', 'PLAQUE TARTE 25P', 'SABLE F  P', 'PAIN SUISSE PEPITO',
    'TULIPE', 'TARTELETTE COCKTAIL', 'SACHET DE VIENNOISERIE'
]

# Clean names: upper case and remove spaces.
clean_product_names = [name.strip().upper() for name in raw_product_names]

# Create a dictionary where each product name is mapped to a unique id.
product_map = {name: idx for idx, name in enumerate(clean_product_names, start=101)}

# Show the mapping information.
print("Product Mapping:")
for product, pid in product_map.items():
    print(f"{product}: {pid}")

# Read the cleaned sales file.
df = pd.read_csv('bakery_sales_merged.csv')

# Check missing values before fixing data.
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Replace missing store info with default values.
df['store_id'] = df['store_id'].fillna(1)
df['store_location'] = df['store_location'].fillna('Bakery - Default Location')

# Clean product details and format the string.
df['product_detail'] = df['product_detail'].astype(str).str.strip().str.upper()

# Ensure product_id exists and fill missing ones with 0.
if 'product_id' not in df.columns:
    df['product_id'] = 0
else:
    df['product_id'] = df['product_id'].fillna(0)

# Map the product details to product IDs using the dictionary.
df['product_id'] = df.apply(
    lambda row: product_map.get(row['product_detail'], 999) if row['product_id'] == 0 else row['product_id'],
    axis=1
)

# Convert transaction ids to integer type.
df['transaction_id'] = df['transaction_id'].astype(int)

# Show missing values after cleaning.
print("\nMissing values after cleaning:")
print(df.isnull().sum())




In [ ]:
unique_stores = Coffee_sales[['store_id', 'store_location']].drop_duplicates()
unique_tickets = bakery_sales['transaction_id'].unique()

# Attribute a random store to each transaction
np.random.seed(42)
ticket_store_mapping = {ticket: np.random.choice(unique_stores['store_id']) for ticket in unique_tickets}

df['store_id'] = bakery_sales['transaction_id'].map(ticket_store_mapping)
df['store_location'] = df['store_id'].map(unique_stores.set_index('store_id')['store_location'])

df.head()

In [ ]:
# Save the cleaned data to a csv file.
output_filename = 'Bakery_sales_cleaned.csv'
df.to_csv(output_filename, index=False)
print(f"\nCleaning complete. Cleaned data saved as '{output_filename}'.")

In [ ]:
df = pd.read_csv("Bakery_sales_cleaned.csv")
df['sales_id'] = range(1, len(df) + 1)
df.to_csv("Bakery_sales_cleaned.csv", index=False)

df2 = pd.read_csv("Coffee_Shop_Sales.csv") 
df2['sales_id'] = range(1, len(df2) + 1)
df2.to_csv("Coffee_Shop_Sales.csv", index=False)

print(df['sales_id'].head())
print(df2['sales_id'].head())

![Local Image](erd.png)


In [ ]:
CREATE TABLE Store (
    store_id INT PRIMARY KEY,
    store_location VARCHAR(255)
);

CREATE TABLE Transactions (
    sale_id INT PRIMARY KEY,
    transaction_id INT,
    transaction_date DATE,
    transaction_time TIME,
    transaction_qty INT,
    store_id INT,
    product_id INT,
    FOREIGN KEY (store_id) REFERENCES Store(store_id),
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
);

CREATE TABLE Product (
    product_id INT PRIMARY KEY,
    product_category VARCHAR(255) NOT NULL,  
    product_type VARCHAR(255),              
    product_detail VARCHAR(255),             
    unit_price DECIMAL(10,2) NOT NULL
);


In [8]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="OntoDb",
    user="ontodb",
    password="admin"
)
cursor = conn.cursor()

cursor.execute("SELECT current_user;")
print("Connected as:", cursor.fetchone())

cursor.execute("SELECT version();")
print("PostgreSQL version:", cursor.fetchone())

cursor.execute("""
CREATE TABLE IF NOT EXISTS Product (
    product_id INT PRIMARY KEY,
    product_category VARCHAR(255) NOT NULL,  
    product_type VARCHAR(255),              
    product_detail VARCHAR(255),             
    unit_price DECIMAL(10,2) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Store (
    store_id INT PRIMARY KEY,
    store_location VARCHAR(255)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    sale_id INT PRIMARY KEY,
    transaction_id INT,
    transaction_date DATE,
    transaction_time TIME,
    transaction_qty INT,
    store_id INT,
    product_id INT,
    FOREIGN KEY (store_id) REFERENCES Store(store_id),
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
);
""")

conn.commit()
cursor.close()
conn.close()


Connected as: ('ontodb',)
PostgreSQL version: ('PostgreSQL 16.6 (Ubuntu 16.6-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0, 64-bit',)


In [9]:
#Followed online tutorial reference : https://www.geeksforgeeks.org/how-to-insert-a-pandas-dataframe-to-an-existing-postgresql-table/


Coffee_sales = pd.read_csv('Coffee_Shop_Sales.csv')

# Write the bakery sales data into the database.
df.to_sql('bakery_sales_data', con=db, if_exists='replace', index=False)

# Also add the Coffee_Shop_Sales.csv data into the database.
Coffee_sales.to_sql('coffee_sales_data', con=db, if_exists='replace', index=False)

# Query and display 5 rows from the bakery sales table.
import psycopg2

conn_db = psycopg2.connect(host="localhost", database="OntoDb", user="ontodb", password="admin")
conn_db.autocommit = True
cursor = conn_db.cursor()

print("Bakery Sales Data (first 5 rows):")
sql_query = "SELECT * FROM bakery_sales_data LIMIT 5;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

print("\nCoffee Shop Sales Data (first 5 rows):")
sql_query_coffee = "SELECT * FROM coffee_sales_data LIMIT 5;"
cursor.execute(sql_query_coffee)
for row in cursor.fetchall():
    print(row)

cursor.close()
conn_db.close()


NameError: name 'df' is not defined

In [ ]:
from sqlalchemy import text

with db.connect() as conn:
    count_query = text("SELECT COUNT(*) FROM bakery_sales_data;")
    total_rows = conn.execute(count_query).scalar()
    print(f"Total rows in bakery sales: {total_rows}")

with db.connect() as conn:
    count_query = text("SELECT COUNT(*) FROM coffee_sales_data;")
    total_rows = conn.execute(count_query).scalar()
    print("Total rows in Coffee sales:", total_rows)